In [3]:
from sklearn import preprocessing,metrics
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
import pandas as pd, numpy as np
import re,nltk, string
from string import punctuation
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


import warnings
warnings.filterwarnings('ignore')

# #### 1.	Load the dataset and create a dataframe.

df=pd.read_excel('ReviewsFileName.xlsx')
df=df.values.tolist()

# #### 2.	Define a function which can perform following functions:
 
# `a.	Remove non-alphabets 
# b.	Remove urls
# c.	Remove digits
# d.	Remove stopwords
# e.	Stem the texts using PorterStemmer
# f.	Remove and replace “’”, “--”, “-”, “[”, “]” by “”.


def remove(df):
    newdf=[]
    regex = re.compile('[^a-zA-Z ]')
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    
    #First parameter is the replacement, second parameter is your input string
    
    for i in range(len(df)):
        temp=df[i][0]
        temp=   re.sub(r'http\S+', '', temp)
        temp = regex.sub('', temp)
        word_tokens = word_tokenize(temp) 
        temp = [ps.stem(w) for w in word_tokens if not w in stop_words]    
        newdf.append([" ".join(temp),df[i][1]])
    return newdf

df=remove(df) 
    

#### Creating a corpus
#### 3.	Create a list of 30 most frequent occurring words from cleaned reviews and write it to 'nlargest.txt'
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize, sent_tokenize
from heapq import nlargest

allwords=[]

for i in range(len(df)):
    temp=df[i][0]
    word_tokens = word_tokenize(temp)
    for j in word_tokens:
        allwords.append(j.lower())

all_words = nltk.FreqDist(allwords)
nlargest=all_words.most_common(30)

with open('nlargest.txt', 'w') as f:
    for item in nlargest:
        f.write("%s\n" % item[0])
    


#### 4.	Create a train (67%) and test (33%) split with random state 42
df=pd.DataFrame(df,columns=['Text','Review'])
X=df['Text']
Xall=X.values.tolist()
y=df['Review']


# #### 5.	Create TF-IDF vector
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(min_df=7, max_df=0.3, ngram_range = (1,2))
Xall=tfidfconverter.fit_transform(Xall).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xall, y, test_size=0.33, random_state=42)


# #### 6.	Build a Random Forest Classifier 

rf=RandomForestClassifier()
rf.fit(X_train,y_train)
rf.predict(Xall)


# #### 7.	After building classification model and predicting on whole dataset, save confusion matrix to text file 

ypred=rf.predict(Xall)
conf=confusion_matrix(y,ypred)
conf.tofile('cfmatrix.txt',sep=',')
